## **OpenMP e GPU no Google Colab**

Para utilizar OpenMP no Google Colab, inicialmente escreva seu código numa célula de código e, na primeira linha, utilize o comando %%writefile com o nome do arquivo a ser gravado no seu diretório de trabalho.

In [ ]:
%%writefile main.cpp

#include <iostream>
int main()
{
    #pragma omp parallel
    {
        std::cout << "Hello World\n";
    }
    return 0;
}

Overwriting main.cpp


Em seguida, você deve compilar o seu código:

In [ ]:
!g++ -fopenmp -o main main.cpp

Para executar o seu código, basta utilizar o nome do executável:

In [ ]:
!./main

Hello World
Hello World


**EXERCÍCIO.** Considere o código abaixo e sua medida de tempo:

In [ ]:
%%writefile main.cpp

#include <iostream>
#define ARRAY_SIZE 100000000
#define ARRAY_VALUE 1231
int main()
{
    int *arr = new int[ARRAY_SIZE];
    std::fill_n(arr, ARRAY_SIZE, ARRAY_VALUE);
    for(int i = 0; i < ARRAY_SIZE; i++)
    {
        // do some relatively long operation
        arr[i] = 3*arr[i]  + arr[i] / 5 - 14;
    }
    return 0;
}

Overwriting main.cpp


In [ ]:
!g++ -o main main.cpp

In [ ]:
!time ./main


real	0m0.868s
user	0m0.663s
sys	0m0.193s


Utilizando OpenMP, paralelize este código e refaça a medida de tempo:

In [ ]:
%%writefile main.cpp

#include <iostream>

#define ARRAY_SIZE 100000000
#define ARRAY_VALUE 1231
int main()
{
    int *arr = new int[ARRAY_SIZE];
    //std::fill_n(arr, ARRAY_SIZE, ARRAY_VALUE);
    int aux  = 3*ARRAY_VALUE+ARRAY_VALUE/5-14;
    #pragma omp parallel for
    for(int i = 0; i < ARRAY_SIZE; i++)
    {
        
        arr[i] = aux;
    }
    return 0;
}

Overwriting main.cpp


In [ ]:
!g++ -fopenmp -o main main.cpp

In [ ]:
!time ./main


real	0m0.660s
user	0m0.791s
sys	0m0.153s


Para utilizar GPU no Google Colab,  você também deve escrever seu código numa célula de código e, na primeira linha, utilize o comando %%writefile com o nome do arquivo a ser gravado no seu diretório de trabalho.

In [ ]:
%%writefile main.cu

#include <thrust/device_vector.h>
#include <iostream>

int main(void)
{
    // inicializa todos os 10 elementos de v na GPU com o valor 1
    thrust::device_vector<int> v(10, 1);
 
    // mostra todos os elementos alocados
    for(int i = 0; i < v.size(); i++)
        std::cout << v[i] << "\n";

    return 0;
}

Writing main.cu


Em seguida, compilamos o arquivo com o nvcc indicando o tipo específico de gpu do Google Colab (sm_37):

In [ ]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o main main.cu

Para executar o programa, primeiro você deve ativar o dispositivo GPU no menu Editar--> Configurações de Notebook --> Acelerador de Hardware (GPU). **ATENÇÃO: DEIXE A GPU ATIVADA SOMENTE ENQUANTO FOR EXECUTAR O PROGRAMA. HÁ UM TEMPO LIMITADO DE USO DAS GPUs NO GOOGLE COLAB.**

In [ ]:
!./main

1
1
1
1
1
1
1
1
1
1


**EXERCÍCIO.** Considere o código C++ abaixo que calcula a norma de um vetor de 4 componentes e suas medidas de tempo:

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <cmath>

int main(void)
{
    // inicializa o vetor 
    float x[4] = {1.0, 2.0, 3.0, 4.0};
    float sum=0.0;
    for (int i=0;i<4;i++)
        sum+=x[i]*x[i];
    std::cout << std::sqrt(sum);
    return 0;
}

Writing main.cpp


In [ ]:
!g++ -o main main.cpp

In [ ]:
!time ./main

5.47723
real	0m0.003s
user	0m0.001s
sys	0m0.002s


Paralelize o código acima em GPU com THRUST e faça uma estimativa de tempo de sua execução:

In [ ]:
%%writefile main.cu
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <cmath>

template <typename T>
struct square
{
    __host__ __device__
        T operator()(const T& x) const { 
            return x * x;
        }
};

int main(void)
{
    // inicializa o vetor
    float x[4] = {1.0, 2.0, 3.0, 4.0};

    // aloca e inicializa o vetor na GPU
    thrust::device_vector<float> d_x(x, x + 4);

    square<float>        unary_op;
    thrust::plus<float> binary_op;
    float init = 0;

    // calcula a norma
    float norm = std::sqrt( thrust::transform_reduce(d_x.begin(), d_x.end(), unary_op, init, binary_op) );

    std::cout << norm << std::endl;

    return 0;
}

Overwriting main.cu


In [ ]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o main main.cu

In [ ]:
!./main

5.47723


In [ ]:
!time ./main


real	0m0.730s
user	0m0.823s
sys	0m0.181s
